In [1]:
import nltk
from sklearn.naive_bayes import MultinomialNB
import xlrd
import string
from nltk.corpus import stopwords
from random import shuffle
import csv
from gensim import corpora

# nltk.download('stopwords')  

In [2]:
Tweet_list = ["realDonaldTrump", "BarackObama"]

In [3]:
def read_in_tweets(handle): 
    contents_all = []
    with open(handle + ".tsv") as f: 
        for l in f.readlines():
            l = l.split('\t')
            if l[0] != 'id':
                contents_all.append(l[-1])
    return(contents_all)

In [14]:
### Remove punctuation for unicode strings (code I got from online)
import regex as re

def remove_punctuation(text):
    return re.sub(u"\p{P}+", "", text)


def data_clean(line):
    l = []
    line = line.split()
    for i in line:
        i = i.lower()
        if i[0] != '@' and i[0] != "#":
            i = remove_punctuation(i)
            if len(i) != 0  and i[0] != '@' and i[:5] != 'https' and i not in stop_words and i != 'rt' and i != 'amp':
                l.append(i)
    return l

<>:5: DeprecationWarning: invalid escape sequence \p
<>:5: DeprecationWarning: invalid escape sequence \p
<>:5: DeprecationWarning: invalid escape sequence \p
<ipython-input-14-7feef1c561eb>:5: DeprecationWarning: invalid escape sequence \p
  return re.sub(u"\p{P}+", "", text)


In [15]:
stop_words = set(stopwords.words('english'))

In [16]:
cleaned_tweets = dict()

for handle in Tweet_list:        
    cleaned_tweets[handle] = [data_clean(l) for l in read_in_tweets(handle)]

In [61]:
from gensim import corpora
import pickle
import gensim
import copy

In [68]:
def get_topics(handle, NUM_TOPICS, NUM_WORDS, NUM_PASSES = 15):
    dictionary = corpora.Dictionary(cleaned_tweets[handle])
    corpus = [dictionary.doc2bow(text) for text in cleaned_tweets[handle]]
    # pickle.dump(corpus, open('corpus/' + handle + '_corpus.pkl', 'wb'))
    # dictionary.save('dictionary/' + hanlde + '_dictionary.gensim')
    
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes = NUM_PASSES)
    topics = ldamodel.print_topics(num_words=NUM_WORDS)
    
    topics_dic = {}
    
    for i in range(NUM_TOPICS):
        topic = topics[i][1].split("+")
        topic_dic = {}
        for item in topic:
            weight = float(item.split("*")[0])
            key = item.split("*")[1].split("\"")[1]
            topic_dic[key] = weight
        topics_dic[i] = topic_dic.copy()
        
    return(topics_dic)

def print_topics(topic_dic):
    for i in topic_dic.keys():
        print("Topic " + str(i + 1) + ":")
        for j in topic_dic[i].keys():
            print('  ' + j + ' (' + str(topic_dic[i][j]) + ')')    

In [ ]:
DT_dic = get_topics("realDonaldTrump", 5, 10)
with open('DT_dic.pkl', 'wb') as f:
    pickle.dump(DT_dic, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
BO_dic = get_topics("BarackObama", 5, 10)
with open('BO_dic.pkl', 'wb') as f:
    pickle.dump(BO_dic, f, pickle.HIGHEST_PROTOCOL)

In [97]:
print_topics(DT_dic)

Topic 1:
  collusion (0.013)
  mueller (0.011)
  years (0.009)
  trump (0.008)
  democrats (0.007)
  never (0.007)
  report (0.007)
  clinton (0.006)
  russia (0.006)
  campaign (0.005)
Topic 2:
  great (0.03)
  thank (0.015)
  border (0.01)
  people (0.01)
  america (0.01)
  wall (0.009)
  country (0.006)
  states (0.006)
  new (0.006)
  united (0.005)
Topic 3:
  party (0.007)
  trump (0.006)
  tonight (0.005)
  pm (0.005)
  great (0.005)
  approval (0.005)
  senate (0.004)
  rating (0.004)
  republican (0.004)
  big (0.004)
Topic 4:
  border (0.011)
  democrats (0.01)
  jobs (0.008)
  president (0.007)
  security (0.007)
  american (0.007)
  economy (0.005)
  years (0.005)
  great (0.005)
  trumps (0.004)
Topic 5:
  president (0.016)
  today (0.01)
  democrats (0.01)
  new (0.008)
  trump (0.008)
  border (0.007)
  done (0.006)
  us (0.006)
  made (0.006)
  great (0.006)


In [98]:
print_topics(BO_dic)

Topic 1:
  president (0.074)
  obama (0.064)
  live (0.011)
  address (0.011)
  american (0.01)
  immigration (0.008)
  speaking (0.008)
  weekly (0.008)
  us (0.007)
  watch (0.007)
Topic 2:
  health (0.02)
  today (0.011)
  president (0.011)
  care (0.01)
  americans (0.009)
  make (0.008)
  obama (0.008)
  time (0.008)
  millions (0.007)
  happy (0.007)
Topic 3:
  president (0.041)
  obama (0.039)
  change (0.02)
  climate (0.017)
  take (0.009)
  health (0.008)
  care (0.008)
  gun (0.006)
  fight (0.006)
  america (0.005)
Topic 4:
  president (0.045)
  obama (0.041)
  watch (0.013)
  et (0.012)
  get (0.011)
  tune (0.01)
  pm (0.009)
  years (0.008)
  americans (0.008)
  today (0.008)
Topic 5:
  senate (0.018)
  name (0.015)
  add (0.014)
  leaders (0.014)
  jobs (0.011)
  judge (0.011)
  supreme (0.011)
  job (0.011)
  court (0.01)
  garland (0.01)


In [88]:
import pyLDAvis.gensim

In [93]:
handle = "BarackObama"

dictionary = corpora.Dictionary(cleaned_tweets[handle])
corpus = [dictionary.doc2bow(text) for text in cleaned_tweets[handle]]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 5, id2word=dictionary, passes = 15)
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)

with open('BO_lda_display.pkl', 'wb') as f:
    pickle.dump(lda_display, f, pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [94]:
handle = "realDonaldTrump"

dictionary = corpora.Dictionary(cleaned_tweets[handle])
corpus = [dictionary.doc2bow(text) for text in cleaned_tweets[handle]]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 5, id2word=dictionary, passes = 15)
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)

with open('DT_lda_display.pkl', 'wb') as f:
    pickle.dump(lda_display, f, pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [95]:
pyLDAvis.display(lda_display)